In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import svm
import math


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dma-fall2020/train.csv
/kaggle/input/dma-fall2020/gender_submission.csv
/kaggle/input/dma-fall2020/test.csv


In [110]:
### Processing training data ###

df_train = pd.read_csv('/kaggle/input/dma-fall2020/train.csv')
cols = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Embarked', 'Parch', 'Fare', 'Survived']
feats = ['Pclass', 'Sex', 'Age', 'SibSp', 'Embarked', 'Parch', 'Fare']
df_train_processed = df_train[cols]
countval = df_train_processed.groupby("Pclass").count()["PassengerId"]
countage = df_train_processed.groupby(["Pclass"]).sum()["Age"]
meanage = np.array(countage/countval)
def replacezeros(row):
    if math.isnan(row["Age"]):
        return meanage.item(int(row["Pclass"]-1))
    else: 
        return row["Age"]
df_train_processed["Age"] = df_train_processed.apply(replacezeros, axis = 1 )
df_train_processed['Embarked'] = df_train_processed['Embarked'].fillna('S')
df_train_processed['Fare'] = df_train_processed['Fare'].fillna(df_train_processed['Fare'].mean())
df_train_processed['Fare'] = df_train_processed['Fare'].astype(int)
df_train_processed.isna().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Embarked       0
Parch          0
Fare           0
Survived       0
dtype: int64

In [111]:
### Train/test split ###

x = df_train_processed[feats]
x['Sex'] = x['Sex'].map({'male': 0, 'female': 1})
x = pd.get_dummies(x)
y = df_train_processed['Survived']
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state=0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [117]:
x_val

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
495,3,0,18.177026,0,0,14,1,0,0
648,3,0,18.177026,0,0,7,0,0,1
278,3,0,7.000000,4,1,29,0,1,0
31,1,1,32.923241,1,0,146,1,0,0
255,3,1,29.000000,0,2,15,1,0,0
...,...,...,...,...,...,...,...,...,...
167,3,1,45.000000,1,4,27,0,0,1
306,1,1,32.923241,0,0,110,1,0,0
379,3,0,19.000000,0,0,7,0,0,1
742,1,1,21.000000,2,2,262,1,0,0


In [122]:
### Decision Tree ###

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
train_predictions_clf = clf.predict(x_val)
accuracy_score(train_predictions_clf, y_val)

0.7937219730941704

In [123]:
### XGBoost Classifier ###

model = XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
train_predictions_xgb = [round(value) for value in y_pred]
accuracy = accuracy_score(y_val, train_predictions_xgb)
print(accuracy)

0.8475336322869955


In [124]:
### SVM Classifier ###

svm_model = svm.SVC(kernel='linear')
svm_model.fit(x_train, y_train)
train_predictions_svm = svm_model.predict(x_val)
accuracy_score(train_predictions_svm, y_val)

0.7802690582959642

In [125]:
### Ensemble ###

ensemble_predictions = []
for i in range(223):
    val = train_predictions_xgb[i] + train_predictions_clf[i] + train_predictions_svm[i]
    if val >= 2:
        ensemble_predictions.append(1)
    else:
        ensemble_predictions.append(0)
accuracy_score(ensemble_predictions, y_val)

0.8430493273542601

In [93]:
### Processing testing data ###

df_test = pd.read_csv('/kaggle/input/dma-fall2020/test.csv')
cols = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Embarked', 'Parch', 'Fare']
df_test_processed = df_test[cols]
countval = df_test_processed.groupby("Pclass").count()["PassengerId"]
countage = df_test_processed.groupby(["Pclass"]).sum()["Age"]
meanage = np.array(countage/countval)
def replacezeros(row):
    if math.isnan(row["Age"]):
        return meanage.item(int(row["Pclass"]-1))
    else: 
        return row["Age"]
df_test_processed["Age"] = df_test_processed.apply(replacezeros, axis = 1 )
df_test_processed['Embarked'] = df_test_processed['Embarked'].fillna('S')
df_test_processed['Fare'] = df_test_processed['Fare'].fillna(df_test_processed['Fare'].mean())
df_test_processed['Fare'] = df_test_processed['Fare'].astype(int)
df_test_processed.isna().sum()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Embarked       0
Parch          0
Fare           0
dtype: int64

In [100]:
### Running each model on test data ###

x_test = df_test_processed.drop("PassengerId", axis=1)
x_test['Sex'] = x_test['Sex'].map({'male': 0, 'female': 1})
x_test = pd.get_dummies(x_test)

model = XGBClassifier()
clf = DecisionTreeClassifier()
svm_model = svm.SVC(kernel='linear')

x_train = df_train_processed[feats]
x_train['Sex'] = x_train['Sex'].map({'male': 0, 'female': 1})
x_train = pd.get_dummies(x_train)
y_train = df_train_processed['Survived']

model.fit(x_train, y_train)
clf.fit(x_train, y_train)
svm_model.fit(x_train, y_train)

y_pred_xgb = model.predict(x_test)
y_pred_clf = clf.predict(x_test)
y_pred_svm = svm_model.predict(x_test)

predictions_xgb = [round(value) for value in y_pred_xgb]
predictions_clf = [round(value) for value in y_pred_clf]
predictions_svm = [round(value) for value in y_pred_svm]
len(predictions_xgb)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


418

In [101]:
print(sum(predictions_xgb))
print(sum(predictions_clf))
print(sum(predictions_svm))

159
161
152


In [102]:
### Ensemble ###

predictions = []
for i in range(418):
    val = predictions_xgb[i] + predictions_clf[i] + predictions_svm[i]
    if val >= 2:
        predictions.append(1)
    else:
        predictions.append(0)
print(sum(predictions))

158


In [106]:
### Generate submission dataframe ###

submission = pd.DataFrame(index=df_test_processed.PassengerId)
submission['Survived'] = predictions

In [107]:
### Check predictions ### 

submission.value_counts()
# 274
# 144
# ensemble - 298, 120
# svm - 266, 152
# svm no gender - 309, 109
# improved - 260, 158

Survived
0           259
1           159
dtype: int64

In [105]:
### Create csv ###

submission.reset_index().to_csv('submission.csv', index=False)